# PROCESO ETL

Importamos Librerias

In [2]:
import pandas as pd
import numpy as np
import re #Modulo de Expresiones Regulares
import unicodedata
import pyarrow as pa
import pyarrow.parquet as pq

## 1 - Extracción de Datos

1.1 - Accedemos a los datos de los archivos movies_dataset.csv y credits.parquet.

In [3]:
#Importamos movies_dataset.csv
moviesDf = pd.read_csv('datasets/movies_dataset.csv', sep=',', encoding='utf-8')
#Importamos credits.csv
#creditsDf = pd.read_csv('datasets/credits.csv', sep=',', encoding='utf-8')
creditsDf = pd.read_parquet('datasets/credits.parquet', engine='pyarrow')

C:\Users\58424\AppData\Local\Temp\ipykernel_18852\97965091.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  moviesDf = pd.read_csv('datasets/movies_dataset.csv', sep=',', encoding='utf-8')


1.2 - Probamos si los DataFrames se importaron correctamente.

In [4]:
moviesDf.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
creditsDf.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


1.3 - Revisamos el tipo de datos de la variables

In [6]:
moviesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [7]:
creditsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


## 2 - Transformación de Datos

2.1 - Extraemos del DataFrame moviesDf los datos de las columnas: belongs_to_collection, genres, production_companies, production_countries, spoken_languages, que se encuentran en formato de listas de diccionarios para poder agregarlos al dataset de forma separada. Adicionalmente extraemos los datos de las columnas: cast, crew del DataFrame creditsDf, y aplicamos las funciones correspondientes.

In [8]:
#Función para extraer y concatenar 'name'
def extract_concatenate_name(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        match = re.findall(pattern, value)
        return ', '.join(match)
    except:
        return ''
    
#re.findall ( )
#Devuelva todas las coincidencias de patrón no superpuestas en cadena, 
# como una lista de cadenas. La cadena se escanea de izquierda a derecha 
# y las coincidencias se devuelven en el orden encontrado.

In [9]:
#Función para extraer y concatenar 'director_name'
def extract_concatenate_director_name(value):
    try:
        pattern = r"'job': 'Director', 'name':\s+'(.*?)'"
        match = re.findall(pattern, value)
        return ', '.join(match)
    except:
        return ''

In [10]:
#Normalizamos la columna "belongs_to_collection"
moviesDf['collection'] = moviesDf['belongs_to_collection'].apply(extract_concatenate_name)

#Normalizamos la columna "genres"
moviesDf['genre'] = moviesDf['genres'].apply(extract_concatenate_name)

#Normalizamos la columna la columna "production_companies"
moviesDf['company'] = moviesDf['production_companies'].apply(extract_concatenate_name)

#Normalizamos la columna "production_countries"
moviesDf['country'] = moviesDf['production_countries'].apply(extract_concatenate_name)

#Normalizamos la columna "spoken_languages"
moviesDf['language'] = moviesDf['spoken_languages'].apply(extract_concatenate_name)

#Normalizamos la columna "cast"
creditsDf['actor'] = creditsDf['cast'].apply(extract_concatenate_name)

#Normalizamos la columna "crew"
creditsDf['director'] = creditsDf['crew'].apply(extract_concatenate_director_name)

In [11]:
#Eliminamos las columnas 'belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'
moviesDf.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace = True)

#Eliminamos las columnas 'cast', 'crew'
creditsDf.drop(columns=['cast', 'crew'], inplace = True)

2.2 - Reemplazamos los valores NA/NaN de las columnas 'budget' y 'revenue' por 0 (Ceros)

In [12]:
moviesDf['budget'].fillna(0, inplace=True)
moviesDf['revenue'].fillna(0, inplace=True)

#Verificamos que realmente no existan valores Na/NaN en los campos tratados
print('# Nulos campo budget:', moviesDf.budget.isna().sum())
print('# Nulos campo revenue:', moviesDf.revenue.isna().sum())

# Nulos campo budget: 0
# Nulos campo revenue: 0


2.3 - Eliminamos los registros con valores Na/NaN de la columna 'release_date'

In [13]:
#Damos formato a todos los valores de la columna 'release_date'
#A los que no se les puede dar formato se les asigna NaT (Not a Time)
moviesDf['release_date'] = pd.to_datetime(moviesDf['release_date'], errors='coerce')

In [14]:
print('Cantidad de Observaciones con fechas nulas:', moviesDf['release_date'].isnull().sum())

Cantidad de Observaciones con fechas nulas: 90


In [15]:
#Eliminamos los registros NaT 
moviesDf.dropna(subset = ['release_date'], inplace=True)

#Verificamor que no existan NaT
print('# Nulos campo release_date:', moviesDf['release_date'].isnull().sum())

# Nulos campo release_date: 0


2.4 - Damos formato AAAA-mm-dd a la columna 'release_date', además creamos la columna 'release_year' donde extraemos el año de la columna 'release_date'.

In [16]:
#Verificamos que todos los registros tengan el mismo formato AAAA-MM-DD
moviesDf['release_date'] = pd.to_datetime(moviesDf['release_date']).dt.strftime('%Y-%m-%d')

#Creamos la columna 'release_year' que va a contener el año de la columna 'release_date' 
moviesDf['release_year'] = pd.to_datetime(moviesDf['release_date']).dt.year

2.5 - Creamos la columna 'return' que contendra el calculo del retorno de la inversion de la pelicula, el cual es el producto de la division entre las columnas 'revenue' y 'budget'.

In [17]:
# Se convierten el tipo de datos de las columnas revenue y budget a int64.
moviesDf['revenue'] = moviesDf['revenue'].astype('int64')
moviesDf['budget'] = moviesDf['budget'].astype('int64')

#Realizamos el calculo del retorno de inversión 'return'
moviesDf['return'] = moviesDf['revenue'] / moviesDf['budget']

In [18]:
#Reemplazamos los valores NaN por el valor 0
moviesDf['return'].fillna(0, inplace=True)

In [19]:
#Reemplazamos los valores "inf" por el valor 0 
moviesDf['return'] = np.where(np.isinf(moviesDf['return']), 0, moviesDf['return'])

2.6 - Eliminamos las columnas innecesarias: 'video', 'imdb_id', 'adult', 'original_title', 'poster_path' y 'homepage'.

In [20]:
moviesDf.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace = True)

### TRANSFORMACIONES ADICIONALES

- Cambiamos el tipo de datos de las columnas 'id' y 'vote_count'

In [21]:
moviesDf['id'] = moviesDf['id'].astype('int64')
moviesDf['vote_count'] = moviesDf['vote_count'].astype('int64')

moviesDf['runtime'].fillna(0, inplace=True)
moviesDf['runtime'] = moviesDf['runtime'].astype('int')

- Creamos las columnas 'release_month' y 'release_day'

In [22]:
#Creamos las columnas release_month, release_day
moviesDf['release_month'] = pd.to_datetime(moviesDf['release_date']).dt.month_name('es_ES')
moviesDf['release_day'] = pd.to_datetime(moviesDf['release_date']).dt.day_name('es_ES')

In [23]:
#Verificamos las columnas release_month, release_day
moviesDf[['release_month','release_day']].head()

,release_month,release_day
0,Octubre,Lunes
1,Diciembre,Viernes
2,Diciembre,Viernes
3,Diciembre,Viernes
4,Febrero,Viernes


- Eliminamos los acentos de la columna 'release_day'

In [24]:
def strip_accents(s): 
    return ''.join(c for c in unicodedata.normalize('NFD', s) 
        if unicodedata.category(c) != 'Mn')
# "Mn" significa Nonspacing_Mark que es similar a unicodedata.combining
# Unidecode es la respuesta correcta para esto. Traduce cualquier cadena unicode a la representación más cercana posible en texto ascii

In [25]:
moviesDf['release_day'] = moviesDf['release_day'].apply(strip_accents)
moviesDf['release_day'].value_counts().sort_values(ascending=True)

release_day
Lunes         3503
Domingo       3613
Martes        4641
Sabado        5151
Miercoles     7035
Jueves        7525
Viernes      13908
Name: count, dtype: int64

- Consultamos los registros duplicados por cada DataFrame

In [26]:
#Duplicados
print('Duplicados movies:', moviesDf.duplicated().sum())
print('Duplicados credits:', creditsDf.duplicated().sum())

# Existen 13 observaciones duplicadas para moviesDf, y 
# 43 observaciones duplicadas para creditsDf

Duplicados movies: 13
Duplicados credits: 43


- Eliminamos los duplicados

In [27]:
moviesDf = moviesDf.drop_duplicates()  # 45363 observaciones
creditsDf = creditsDf.drop_duplicates() # 45433 observaciones

- Verificamos los registros repetidos por 'id'

In [28]:
#Duplicados
print('ID Duplicados movies:', moviesDf['id'].duplicated().sum())
print('ID Duplicados credits:', creditsDf['id'].duplicated().sum())

# Existen 17 observaciones duplicadas para moviesDf, y 
# 1 observacion duplicada para creditsDf

ID Duplicados movies: 17
ID Duplicados credits: 1


- Eliminamos los registros con 'id' duplicados

In [29]:
moviesDf = moviesDf.drop_duplicates(subset='id') #45346 observaciones
creditsDf = creditsDf.drop_duplicates(subset='id') #45432 observaciones

- Hacemos merge entre los dos DataFrames

In [30]:
moviesCreditsDf = pd.merge(moviesDf, creditsDf, on='id', how='left')

In [31]:
moviesCreditsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45346 non-null  int64  
 1   id                 45346 non-null  int64  
 2   original_language  45335 non-null  object 
 3   overview           44405 non-null  object 
 4   popularity         45346 non-null  object 
 5   release_date       45346 non-null  object 
 6   revenue            45346 non-null  int64  
 7   runtime            45346 non-null  int32  
 8   status             45266 non-null  object 
 9   tagline            20387 non-null  object 
 10  title              45346 non-null  object 
 11  vote_average       45346 non-null  float64
 12  vote_count         45346 non-null  int64  
 13  collection         45346 non-null  object 
 14  genre              45346 non-null  object 
 15  company            45346 non-null  object 
 16  country            453

- Damos formatos a las variables categoricas

In [32]:
moviesCreditsDf['title'] = moviesCreditsDf['title'].str.lower()
moviesCreditsDf['tagline'] = moviesCreditsDf['tagline'].str.lower()
moviesCreditsDf['overview'] = moviesCreditsDf['overview'].str.lower()
moviesCreditsDf['collection'] = moviesCreditsDf['collection'].str.lower()
moviesCreditsDf['genre'] = moviesCreditsDf['genre'].str.lower()
moviesCreditsDf['company'] = moviesCreditsDf['company'].str.lower()
moviesCreditsDf['original_language'] = moviesCreditsDf['original_language'].str.lower()
moviesCreditsDf['status'] = moviesCreditsDf['status'].str.lower()
moviesCreditsDf['country'] = moviesCreditsDf['country'].str.lower()
moviesCreditsDf['language'] = moviesCreditsDf['language'].str.lower()
moviesCreditsDf['release_month'] = moviesCreditsDf['release_month'].str.lower()
moviesCreditsDf['release_day'] = moviesCreditsDf['release_day'].str.lower()
moviesCreditsDf['actor'] = moviesCreditsDf['actor'].str.lower()
moviesCreditsDf['director'] = moviesCreditsDf['director'].str.lower()

In [33]:
moviesCreditsDf.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genre,company,country,language,release_year,return,release_month,release_day,actor,director
0,30000000,862,en,"led by woody, andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,released,NaN,...,"animation, comedy, family",pixar animation studios,united states of america,english,1995,12.451801,octubre,lunes,"tom hanks, tim allen, don rickles, jim varney,...",john lasseter


## 3 - Carga de Datos

3.1 - Organizamos las columnas

In [34]:
moviesCreditsDf = moviesCreditsDf[['id','title','tagline','overview','collection','genre','company',
          'actor','director','original_language','status','country','language','runtime','popularity',
          'vote_count','vote_average','release_date','release_month','release_day','release_year',
          'revenue','budget','return']]

3.2 - Exportamos a un archivo .csv

In [35]:
moviesCreditsDf.to_csv('datasets/etl.csv', index=False, header=True, sep=';', encoding='utf-8')
#45346 Observaciones

In [36]:
moviesCreditsDf.shape

(45346, 24)